# Notebook to demonstrate plotting outlines of footprint regions

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import bokeh
import schedview.compute.footprint
import schedview.plot.footprint
from rubin_scheduler.scheduler.utils import get_current_footprint
from uranography.api import (
    Planisphere,
    MollweideMap,
    ArmillarySphere,
)

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

Load the notebook regions, get the polygons, and show the `DataFrame` returned:

In [ ]:
nside=64
footprint_regions = get_current_footprint(nside)[1]
footprint_polygons = schedview.compute.footprint.find_healpix_area_polygons(footprint_regions)
footprint_polygons

The "`loop`" index is there to support occasions where regions are composed of multiple distincte polygons, for example the separete areas of `lowdust` in the North and South Galactic Caps. (See the example below.)

The `x`, `y`, and `z` columns are the "vector" representation used in healpy, e.g. in [`hp.arg2vec`](https://healpy.readthedocs.io/en/latest/generated/healpy.pixelfunc.ang2vec.html#healpy.pixelfunc.ang2vec). This is particularly useful if you want to manipulate the polygons with `shapely`, which seems to handle 3 dimensional Cartesion coordinates better than spherical coordinates.

Now, plot the footprint outlines:

In [ ]:
psphere = Planisphere()
asphere = ArmillarySphere()
msphere = MollweideMap()
schedview.plot.footprint.add_footprint_outlines_to_skymaps(footprint_polygons, [psphere, asphere, msphere], line_width=5)
psphere.add_graticules(label_ra=True, label_decl=False)
asphere.add_graticules()
msphere.add_graticules()
bokeh.io.show(bokeh.layouts.row([psphere.figure, asphere.figure, msphere.figure]))

We can also just plot specific regions. When plotting multiple regions, just index by them, but when plotting one, pandas takes out the index you indexed by, so you need to put it back:

In [ ]:
lowdust_polygons = footprint_polygons.loc['lowdust', :]
psphere = Planisphere()
asphere = ArmillarySphere()
msphere = MollweideMap()
schedview.plot.footprint.add_footprint_outlines_to_skymaps(lowdust_polygons, [psphere, asphere, msphere], line_width=5)
psphere.add_graticules(label_ra=True, label_decl=False)
asphere.add_graticules()
msphere.add_graticules()
bokeh.io.show(bokeh.layouts.row([psphere.figure, asphere.figure, msphere.figure]))


Plotting with outlines works well in uranography only when the polygon does not cross a discontinuity in the map projection, because it isn't smart enough to handle the discontinuities.

For the LSST footprint, this means it works great for azimuthal projections such as the Lambert Azimuthal Equal Area one, but not so well for orthographic or Mollweide.

In [ ]:
psphere = Planisphere()
psphere.plot.title="Lambert Azimuthal Equal Area projection"
asphere = ArmillarySphere()
asphere.plot.title="Dynamic Orthographic projection"
msphere = MollweideMap()
msphere.plot.title="Mollweide projection"
schedview.plot.footprint.add_footprint_outlines_to_skymaps(footprint_polygons, [psphere, asphere, msphere], filled=True)
psphere.add_graticules(label_ra=True, label_decl=False)
asphere.add_graticules()
msphere.add_graticules()
bokeh.io.show(bokeh.layouts.row([psphere.figure, asphere.figure, msphere.figure]))